In [1]:
from qiskit_nature.drivers import PySCFDriver

molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
driver = PySCFDriver(atom=molecule)
qmolecule = driver.run()

from qiskit_nature.transformers import FreezeCoreTransformer
freezeCoreTransformer = FreezeCoreTransformer()
#qmolecule = freezeCoreTransformer.transform(qmolecule)


n_el = qmolecule.num_alpha + qmolecule.num_beta

n_mo = qmolecule.num_molecular_orbitals

n_so = 2 * qmolecule.num_molecular_orbitals

n_q = 2* qmolecule.num_molecular_orbitals

e_nn = qmolecule.nuclear_repulsion_energy

print(n_el,n_mo,n_so,n_q,e_nn)

from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
#problem = ElectronicStructureProblem(driver,q_molecule_transformers=[FreezeCoreTransformer(freeze_core=True)])
problem = ElectronicStructureProblem(driver,q_molecule_transformers=[FreezeCoreTransformer(freeze_core=True,remove_orbitals=[4,3])])
#problem = ElectronicStructureProblem(driver)
# Generate the second-quantized operators
second_q_ops = problem.second_q_ops()
# Hamiltonian
main_op = second_q_ops[0]


from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
#mapper = JordanWignerMapper()
mapper = ParityMapper()
#converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)
converter = QubitConverter(mapper=mapper, two_qubit_reduction=True, z2symmetry_reduction=[1,1])
# The fermionic operators are mapped to qubit operators
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
print (num_particles)
qubit_op = converter.convert(main_op, num_particles=num_particles)


from qiskit_nature.circuit.library import HartreeFock
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
print(init_state)



# from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
# from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
# import numpy as np 

# def exact_diagonalizer(problem, converter):
#     solver = NumPyMinimumEigensolverFactory()
#     calc = GroundStateEigensolver(converter, solver)
#     result = calc.solve(problem)
#     return result

# result_exact = exact_diagonalizer(problem, converter)
# exact_energy = np.real(result_exact.eigenenergies[0])
# print("Exact electronic energy", exact_energy)
# print(result_exact)









from qiskit.circuit.library import TwoLocal
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD
ansatz_type = "TwoLocal"
# Parameters for q-UCC antatze
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals
# Put arguments for twolocal
if ansatz_type == "TwoLocal":
    # Single qubit rotations that are placed on all qubits with independent parameters
    rotation_blocks = ['ry', 'rz']
    #rotation_blocks = ['ry','rz','rx']
    # Entangling gates
    #entanglement_blocks = 'cx'
    entanglement_blocks = ['cx']
    # How the qubits are entangled 
    #entanglement = 'full'
    entanglement = 'linear'
    #entangler_map = [[0,1],[1,2],[2,3]]
    #entangler_map = [[(0,1)],[(2,3),(1,0)]]
    # Repetitions of rotation_blocks + entanglement_blocks with independent parameters
    #repetitions = 3
    #repetitions = 2
    repetitions = 1
    # Skip the final rotation_blocks layer
    #skip_final_rotation_layer = True
    skip_final_rotation_layer = False
    skip_unentangled_qubits = False
    #ansatz = TwoLocal(qubit_op.num_qubits, rotation_blocks, entanglement_blocks, entangler_map, reps=repetitions, skip_final_rotation_layer=skip_final_rotation_layer,skip_unentangled_qubits=skip_unentangled_qubits)
    ansatz = TwoLocal(qubit_op.num_qubits, rotation_blocks, entanglement_blocks, reps=repetitions, entanglement=entanglement, skip_final_rotation_layer=skip_final_rotation_layer,skip_unentangled_qubits=skip_unentangled_qubits)
    # Add the initial state
    ansatz.compose(init_state, front=True, inplace=True)
elif ansatz_type == "UCCSD":
    ansatz = UCCSD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "PUCCD":
    ansatz = PUCCD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "SUCCD":
    ansatz = SUCCD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "Custom":
    # Example of how to write your own circuit
    from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister
    # Define the variational parameter
    theta = Parameter('a')
    n = qubit_op.num_qubits
    # Make an empty quantum circuit
    qc = QuantumCircuit(qubit_op.num_qubits)
    qubit_label = 0
    # Place a Hadamard gate
    qc.h(qubit_label)
    # Place a CNOT ladder
    for i in range(n-1):
        qc.cx(i, i+1)
    # Visual separator
    qc.barrier()
    # rz rotations on all qubits
    qc.rz(theta, range(n))
    ansatz = qc
    ansatz.compose(init_state, front=True, inplace=True)

print(ansatz)
#ansatz.draw('mpl')
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
optimizer_type = 'SLSQP'
# You may want to tune the parameters 
# of each optimizer, here the defaults are used
if optimizer_type == 'COBYLA':
    optimizer = COBYLA(maxiter=5000)
elif optimizer_type == 'L_BFGS_B':
    optimizer = L_BFGS_B(maxfun=5000)
elif optimizer_type == 'SPSA':
    optimizer = SPSA(maxiter=5000)
elif optimizer_type == 'SLSQP':
    optimizer = SLSQP(maxiter=5000)



    
    
    
    
from qiskit.algorithms import VQE
from IPython.display import display, clear_output
# Print and save the data in lists
def callback(eval_count, parameters, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []

# Set initial parameters of the ansatz
# We choose a fixed small displacement 
# So all participants start from similar starting point
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(qubit_op)

print(result)

ModuleNotFoundError: No module named 'qiskit_nature'